## **<u>CSCI 381: PHW#4 - Individual (Marcus Gale)</u>**

#### Chapter 8

\[ChatGPT \*new\*  Generates Propositions\]([https://chatgpt.com/share/671e9f3a-a014-800e-a89c-c0512b3b07c3](https://chatgpt.com/share/671e86d9-1608-800e-9776-683e34ba4ea0))

### 1. **Insert a New Product into Inventory**

Write a query to add a new product to the `StockItems` table with specific details such as `StockItemName`, `SupplierID`, `UnitPrice`, `TaxRate`, and `StockHolding`. This query should simulate the addition of a new item to the inventory.

In [ ]:
INSERT INTO Warehouse.StockItems (StockItemName, SupplierID, UnitPrice, TaxRate, StockHolding)
VALUES ('New Product Name', 5, 15.99, 0.2, 100);

### 2. **Insert Qualified Customer Leads into Customers Table**

Write a query to insert customer data from a temporary or staging table into the `Customers` table. Filter for only the qualified leads (e.g., `IsQualified = 1`) by selecting from a temporary table that stores new customer leads.

In [ ]:
INSERT INTO Sales.Customers (CustomerName, BillToCustomerID, CustomerCategoryID, PrimaryContactPersonID, DeliveryMethodID, PhoneNumber, FaxNumber, DeliveryAddressLine1, DeliveryCityID, AccountOpenedDate)
SELECT LeadName, LeadID, 2, 1, 3, LeadPhone, LeadFax, LeadAddress, LeadCityID, GETDATE()
FROM Sales.CustomerLeads
WHERE IsQualified = 1;

### 3. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"><b style="font-size: 14px;">Insert a Customer Using Stored Procedure</b><span style="font-size: 14px;"><b><br></b></span></span><span style="font-size: 14px;"><b><br></b></span>Write a query using an `EXEC` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> statement to run a stored procedure (e.g., </span> `uspAddNewCustomer`<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">) to add a new customer to the </span> `Customers` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table. The procedure should take dynamic values such as </span> `CustomerName` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> and </span> `PrimaryContact`<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">.</span>

In [ ]:
DECLARE @CustomerName NVARCHAR(100) = 'New Customer', 
        @PrimaryContact NVARCHAR(50) = 'John Doe', 
        @Phone NVARCHAR(20) = '123-456-7890';

EXEC Sales.uspAddNewCustomer @CustomerName, @PrimaryContact, @Phone;

### 4. **Delete Inactive Customer Orders**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Write a query to delete orders from the </span> `Orders` <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table that are associated with inactive customers. Use a join between the </span> `Orders` <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table and </span> `Customers` <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table, applying a condition like </span> `IsActive = 0`<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">.</span>

In [ ]:
DELETE o
FROM Sales.Orders AS o
INNER JOIN Sales.Customers AS c ON o.CustomerID = c.CustomerID
WHERE c.IsActive = 0;

### 5. **Delete Discontinued Products Without Transactions**

Write a query to delete products from the `StockItems` table that are discontinued and have no associated transactions. This query should involve a join between the `StockItems` table and the `OrderLines` table to identify products without orders.

In [ ]:
DELETE si
FROM Warehouse.StockItems AS si
LEFT JOIN Sales.OrderLines AS ol ON si.StockItemID = ol.StockItemID
WHERE si.IsDiscontinued = 1 AND ol.StockItemID IS NULL;

## 6. **Truncate the Special Deals Table**
- - Write a query to truncate the `SpecialDeals` table, removing all existing records after a yearly promotion reset. This truncation will clear the table without logging individual deletions.

In [ ]:
TRUNCATE TABLE Sales.SpecialDeals;

## 7. **Update Tax Rate for Products in a Specific Category**
Write a query to update the `StockItems` table to set a new tax rate for all products in a particular category. Join the `StockItems` table with the `StockItemStockGroups` table to filter by category.

In [ ]:
UPDATE si
SET TaxRate = 0.15
FROM Warehouse.StockItems AS si
INNER JOIN Warehouse.StockItemStockGroups AS sisg ON si.StockItemID = sisg.StockItemID
WHERE sisg.StockGroupID = 3;  -- Assume category is represented by StockGroupID 3

## <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">8.&nbsp;</span> **Merge Customer Data for Upsert Operation  
**
<span style="font-size: 14px;">W</span>rite a merge query to update or insert customer data from a staging table (e.g., `StagingCustomers`<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">) into the </span> `Customers` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table. Use </span> `CustomerID` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> as the key to either update existing customer details (like </span> `PhoneNumber` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> and </span> `EmailAddress`<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">) or insert new customers if they do not already exist.</span>

In [ ]:
MERGE Sales.Customers AS target
USING Staging.StagingCustomers AS source
ON target.CustomerID = source.CustomerID
WHEN MATCHED THEN 
    UPDATE SET PhoneNumber = source.PhoneNumber, 
               EmailAddress = source.EmailAddress
WHEN NOT MATCHED THEN 
    INSERT (CustomerID, CustomerName, PhoneNumber, EmailAddress)
    VALUES (source.CustomerID, source.CustomerName, source.PhoneNumber, source.EmailAddress);

## 9.  Synchronize Inventory Counts in StockItems Table  
<span style="font-size: 14px;">Wr</span>ite a merge query to synchronize inventory data in the `StockItems` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table with data from an </span> `InventoryCounts` <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table. Update stock quantities for existing items, insert any new items not present in </span> `StockItems`<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">, and delete items no longer in </span> `InventoryCounts`

In [ ]:
MERGE Warehouse.StockItems AS target
USING Inventory.InventoryCounts AS source
ON target.StockItemID = source.StockItemID
WHEN MATCHED THEN 
    UPDATE SET StockQuantity = source.StockQuantity
WHEN NOT MATCHED BY TARGET THEN 
    INSERT (StockItemID, StockItemName, StockQuantity)
    VALUES (source.StockItemID, source.StockItemName, source.StockQuantity)
WHEN NOT MATCHED BY SOURCE THEN 
    DELETE;

## 10. **Merge External Orders Based on Status Changes**

- <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Write a merge query to integrate order data from an external table (e.g., </span> `ExternalOrders`<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">) into the </span> `Orders` <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> table. Update existing orders only if the </span> `OrderStatus` <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> has changed, insert new orders, and mark any canceled orders for further review.</span>

In [ ]:
MERGE Sales.Orders AS target
USING External.ExternalOrders AS source
ON target.OrderID = source.OrderID
WHEN MATCHED AND target.OrderStatus <> source.OrderStatus THEN 
    UPDATE SET OrderStatus = source.OrderStatus
WHEN NOT MATCHED BY TARGET THEN 
    INSERT (OrderID, CustomerID, OrderDate, OrderStatus)
    VALUES (source.OrderID, source.CustomerID, source.OrderDate, source.OrderStatus)
WHEN NOT MATCHED BY SOURCE THEN 
    UPDATE SET OrderStatus = 'Marked for Review';